In [64]:
run sentimentanalysis.py

Loading dataset
2000 documents


In [7]:
import math
# from multiprocessing import Pool

In [65]:
texts[0].split();

In [66]:
def count_word(text):
    words_text = text.split()
    unique, count = np.unique(words_text, return_counts=True)
    return pd.Series(dict(zip(unique, count)))

In [70]:
count_word(texts[0])

!            3
"           10
&            1
(            9
)            9
,           44
-            7
.           34
10/10        2
2            1
20           1
3            1
4/10         1
7/10         2
8/10         1
9/10         2
:            3
?            6
a           14
about        2
accident     1
actors       1
actually     2
after        2
again        2
ago          1
all          6
also         1
although     1
always       1
            ..
up           2
us           1
very         2
video        1
visions      1
want         1
watch        1
way          2
ways         1
we           4
weird        1
well         1
wes          1
what         2
what's       2
whatever     1
when         1
where's      1
which        4
while        1
who          3
witch        1
with         5
world        2
would        1
wrapped      1
write        1
years        1
you          3
your         2
Length: 353, dtype: int64

In [67]:
def count_words(texts, count_word=count_word):
    data = map(count_word, texts)
    concat = pd.concat(data, axis = 1, ignore_index=True, sort=False)
    return concat

In [68]:
count_words(texts);

In [5]:
class NB(BaseEstimator, ClassifierMixin):
    def __init__(self, count_words=count_words):
        self._count_words = count_words

    def fit(self, texts, y):
        concat = self._count_words(texts)
        self.vocab = {k:v for v, k in enumerate(list(concat.index))}
        V = set(concat.index)
        N = len(texts)
        self.C, counts = np.unique(y, return_counts=True)
        self.prior = {}
        self.condprob = {}
        for classe in self.C:
            Nc = counts[classe]
            self.prior[classe] = Nc / N
            text_c = concat.iloc[:,y==classe]
            count = text_c.sum(axis=1)
            self.condprob[classe] = (count+1)/sum(count+1)
        return V, self.prior, self.condprob

    def _predict_(self, text):
        W = text.split()
        score = {}
        for classe in self.C:
            score[classe]= math.log(self.prior[classe])
            for t in W:
                try:
                    score[classe]+= math.log(self.condprob[classe][self.vocab[t]])
                except:
                    pass
        return max(score, key=score.get)
    
    def predict(self, texts):
        return list(map(self._predict_, texts))

    def score(self, X, y):
        pred = self.predict(X)
        return np.mean( np.array(pred) == np.array(y))

In [6]:
nb = NB()
nb.fit(texts,y)

({'potty',
  'him/herself',
  'half-stated',
  'vaporize',
  'terse',
  'whos',
  'brash',
  'chelcie',
  'debris',
  'aa',
  'shoot-ups',
  "gosse's",
  'yulin',
  'seing',
  'shread',
  'spoofy',
  'carolco',
  'graveyards',
  'danced',
  'sikh',
  'paraplegic',
  'lan',
  'bull-crap',
  'spielberg-inspired',
  'suitably',
  'trenchcoat',
  "tulips'",
  "material's",
  'captor',
  'capably',
  "hogarth's",
  'muslim',
  '`bring',
  'egos',
  'viruses',
  "garicia's",
  'slake',
  'mistrustful',
  'bubble',
  'pseudo-epic',
  'reville',
  'toast',
  'chunnel',
  'liner',
  'wallet',
  'percussive',
  'chauffeur-driven',
  'hardly',
  'headliner',
  'yielded',
  'before---and',
  'hopping',
  "what're",
  '29-year-old',
  "'amateur'",
  'afterthought',
  'tale',
  'gray',
  'critic',
  'imbecilic',
  'amassed',
  'awkward',
  'anakin',
  'squeaky',
  'crotchety',
  'tracer',
  'non-fiction',
  'impart',
  'familiar',
  'already-chaotic',
  'freakish',
  "gadget's",
  'katy',
  'droopy-

In [20]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)
SCORE = []
for train_index, test_index in kf.split(range(2000)):
    X_train, X_test = [texts[i] for i in train_index], [texts[i] for i in test_index]
    y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
    nb = NB()
    nb.fit(X_train, y_train)
    SCORE.append(nb.score(X_test, y_test))
    break
print(np.mean(SCORE))

0.8125


In [9]:
with open("./34data/data/english.stop", encoding="utf8", mode="r+") as f:
    stop_words = f.read().splitlines()

In [21]:
def count_word(text):
    words_text = text.split()
    words_without_stop_words = [word for word in words_text if word not in stop_words]
    unique, count = np.unique(words_without_stop_words, return_counts=True)
    return pd.Series(dict(zip(unique, count)))

In [22]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)
SCORE = []
for train_index, test_index in kf.split(range(2000)):
    X_train, X_test = [texts[i] for i in train_index], [texts[i] for i in test_index]
    y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
    nb = NB()
    nb.fit(X_train, y_train)
    SCORE.append(nb.score(X_test, y_test))
print(np.mean(SCORE))

0.8155000000000001


In [47]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [48]:
cv=CountVectorizer('content')

In [63]:
cv.fit(texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [42]:
cv.vocabulary

In [49]:
from sklearn.feature_extraction.text import TfidfTransformer

In [59]:
from sklearn.linear_model import LogisticRegression

In [54]:
stemmer = SnowballStemmer('english')

In [57]:
# CountVectorizer(stop_words=stop_words)
stem_vectorizer = StemmedCountVectorizer(stemmer)

In [60]:
text_clf = Pipeline([
    ('vect', stem_vectorizer),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression())
])

In [84]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)
SCORE = []
for train_index, test_index in kf.split(range(2000)):
    X_train, X_test = [texts[i] for i in train_index], [texts[i] for i in test_index]
    y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
    text_clf.fit(X_train, y_train) 
    SCORE.append(text_clf.score(X_test, y_test))
print(np.mean(SCORE))

0.826


In [53]:
from nltk import SnowballStemmer

In [56]:
class StemmedCountVectorizer(CountVectorizer):
    def __init__(self, stemmer):
        super(StemmedCountVectorizer, self).__init__()
        self.stemmer = stemmer

    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc:(self.stemmer.stem(w) for w in analyzer(doc))

In [79]:
class stemmer():
    def __init__(self):
        
        pass
    def fit(self, *args):
        return self
    def transform(self,words):
        stem = SnowballStemmer("english")
        return [stem.stem(word) for word in words]

In [39]:
stemmer.stem("!")

'!'

In [44]:
def count_word(text):
    words_text = text.split()
    words_stem = [stemmer.stem(word) for word in words_text if word not in stop_words]
    unique, count = np.unique(words_stem, return_counts=True)
    return pd.Series(dict(zip(unique, count)))

In [44]:
def count_word(text):
    words_text = text.split()
    words_f = [stemmer.stem(word) for word in words_text if (word not in stop_words) and (pos_tag([word])[0][1] in ['NN', 'ADV','VB','ADJ'])]
    unique, count = np.unique(words_stem, return_counts=True)
    return pd.Series(dict(zip(unique, count)))

In [10]:
from nltk import pos_tag

In [42]:
pos_tag(["buy", "you"])

[('buy', 'NN'), ('you', 'PRP')]

In [43]:
pos_tag(["buy"])
pos_tag(["you"])

[('you', 'PRP')]

In [35]:
import nltk

In [61]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)
SCORE = []
for train_index, test_index in kf.split(range(2000)):
    X_train, X_test = [texts[i] for i in train_index], [texts[i] for i in test_index]
    y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
    text_clf.fit(X_train, y_train) 
    SCORE.append(text_clf.score(X_test, y_test))
print(np.mean(SCORE))

/home/b/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8265
